In [1]:
import cv2
import os
import numpy as np

In [2]:
dirname = 'lfwcrop_color/faces/'
filenames = os.listdir(dirname)

In [14]:
def load_imgs(dirname, filenames):
    imgs = []
    for fname in filenames:
        try:
            im = cv2.imread(dirname + fname)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            im = np.reshape(im, 64 * 64)
        except:
            print fname
            raise
        imgs.append(im)
    imgs = np.stack(imgs, axis = 0)
    return imgs

In [15]:
from sklearn.cross_validation import train_test_split
from random import shuffle

shuffle(filenames)
target_names = [' '.join(f.split('_')[:-1]) for f in filenames]

class Counting(object):
    n = 0
    def factory():
        Counting.n += 1
        return Counting.n
    factory = staticmethod(factory)

from collections import defaultdict
names_id = defaultdict(Counting.factory)

y = [names_id[t] for t in target_names] 

In [18]:
#X_train, X_test, y_train, y_test = train_test_split(filenames, y, test_size=0.2)
#y_train = load_imgs(dirname, X_train)
#y_test = load_imgs(dirname, X_test)

X = load_imgs(dirname, filenames)

.DS_Store


error: /tmp/opencv20160107-29960-t5glvv/opencv-2.4.12/modules/imgproc/src/color.cpp:3739: error: (-215) scn == 3 || scn == 4 in function cvtColor


# PCA

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
def center(faces):
    n_samples, n_features = faces.shape
    # Center the faces
    faces_centered = faces - faces.mean(axis=0)
    faces_centered -= faces_centered.mean(axis=1).reshape(n_samples, -1)
    return faces_centered

faces_centered = center(I_train)

In [ ]:
n_components = 150

In [ ]:
from sklearn import decomposition

In [ ]:
# We use a variant of PCA called Randomized PCA for efficiency. It uses stochastic SVD.
print("Extracting the top %d eigenfaces from %d faces" % (n_components, X.shape[0]))
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_pca = pca.transform(X)
print("done in %0.3fs" % (time() - t0))
print(X_pca.shape)
print(X_pza)